# モデルを考察する（２）

あらかじめ、試行する _k_<sub>_4_</sub>, _k_<sub>_6_</sub> の組み合わせを決め、シミュレーションを実行してグラフを出力するプログラムの例です。


### 準備作業

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import biosim_course

In [ ]:
EM = '''
Stepper ODEStepper( ODE ) {}
Stepper PassiveStepper( PSV ) {}

System System( / )
{
    StepperID  ODE;
    Name  default;
}

System System( /Cell )
{
    StepperID  ODE;

    Variable Variable( SIZE )
    {
        Value  1.0;
    }
    
    Variable Variable( C2 )
    {
        Name  cdc2k;
        Value  0.0;
    }
    
    Variable Variable( CP )
    {
        Name  "cdc2k-P";
        Value  0.75;
    }
    
    Variable Variable( M )
    {
        Name  "p-cyclin_cdc2";
        Value  0.0;
    }
    
    Variable Variable( pM )
    {
        Name  "p-cyclin_cdc2-p";
        Value  0.25;
    }
    
    Variable Variable( Y )
    {
        Name  cyclin;
        Value  0.0;
    }
    
    Variable Variable( YP )
    {
        Name  "p-cyclin";
        Value  0.0;
    }
    
    Variable Variable( YT )
    {
        Name  total_cyclin;
        Value  0.25;
    }
    
    Variable Variable( CT )
    {
        Name  total_cdc2;
        Value  1.0;
    }

### Reactions ###

    Process ExpressionFluxProcess( R1 )
    {
        Name  "cyclin biosynthesis";
        k1aa  0.015;
        Expression  "k1aa / CT.Value";
        VariableReferenceList
            [ Y    :.:Y   1 ]
            [ CT   :.:CT  0 ];
    }
    
    Process ExpressionFluxProcess( R2 )
    {
        Name  "default degradation of cyclin";
        k2  0.0;
        Expression  "k2 * Y.Value";
        VariableReferenceList
            [ Y   :.:Y  -1 ];
    }
    
    Process ExpressionFluxProcess( R3 )
    {
        Name  "cyclin cdc2k-p association";
        k3  200.0;
        Expression  "k3 * CP.Value * Y.Value / CT.Value";
        VariableReferenceList
            [ CP   :.:CP  -1 ]
            [ Y    :.:Y   -1 ]
            [ pM   :.:pM   1 ]
            [ CT   :.:CT   0 ];
    }
    
    Process ExpressionFluxProcess( R4 )
    {
        Name  "activation of cdc2 kinase";
        k4  180.0;
        k4prime  0.018;
        Expression  "pM.Value * (k4prime + k4 * pow(M.Value / CT.Value, 2))";
        VariableReferenceList
            [ pM   :.:pM  -1 ]
            [ M    :.:M    1 ]
            [ CT   :.:CT   0 ];
    }
    
    Process ExpressionFluxProcess( R5 )
    {
        Name  "deactivation of cdc2 kinase";
        k5_P  0.0;
        Expression  "k5_P * M.Value";
        VariableReferenceList
            [ M    :.:M   -1 ]
            [ pM   :.:pM   1 ];
    }
    
    Process ExpressionFluxProcess( R6 )
    {
        Name  "cyclin_cdc2k dissociation";
        k6  1.0;
        Expression  "k6 * M.Value";
        VariableReferenceList
            [ M    :.:M   -1 ]
            [ C2   :.:C2   1 ]
            [ YP   :.:YP   1 ];
    }
    
    Process ExpressionFluxProcess( R7 )
    {
        Name  "cdc2 kinase triggered degration of cyclin";
        k7  0.6;
        Expression  "k7 * YP.Value";
        VariableReferenceList
            [ YP   :.:YP  -1 ];
    }
    
    Process ExpressionFluxProcess( R8 )
    {
        Name  "cdc2k phosphorylation";
        k8_P  1000000.0;
        Expression  "C2.Value * k8_P";
        VariableReferenceList
            [ C2   :.:C2  -1 ]
            [ CP   :.:CP   1 ];
    }
    
    Process ExpressionFluxProcess( R9 )
    {
        Name  "cdc2k dephosphorylation";
        k9  1000.0;
        Expression  "CP.Value * k9";
        VariableReferenceList
            [ CP   :.:CP  -1 ]
            [ C2   :.:C2   1 ];
    }

### Assignment total ammounts ###

    Process ExpressionAssignmentProcess( YT )
    {
        StepperID  PSV;
        Name  "Assignment rule for 'YT'";
        Expression  "Y.Value + YP.Value + M.Value + pM.Value";
        VariableReferenceList
            [ YT :.:YT  1 ]
            [ Y  :.:Y   0 ]
            [ YP :.:YP  0 ]
            [ M  :.:M   0 ]
            [ pM :.:pM  0 ];
    }
    
    Process ExpressionAssignmentProcess( CT )
    {
        StepperID  PSV;
        Name  "Assignment rule for 'CT'";
        Expression  "C2.Value + CP.Value + M.Value + pM.Value";
        VariableReferenceList
            [ CT :.:CT  1 ]
            [ C2 :.:C2  0 ]
            [ CP :.:CP  0 ]
            [ M  :.:M   0 ]
            [ pM :.:pM  0 ];
    }

}
'''



In [ ]:
setModel( EM, "Tyson1991")
print 't = {}'.format( getCurrentTime() )

### システムパス `/cell` 内のすべての `Variable`, `Process` の `LoggerStub` を作成し、`create()` する

In [ ]:
target_SystemPath_list = ( '/Cell', )
Target_Properties = { 'Variable': ['Value', 'Velocity'], 'Process': ['Activity'] }
Logger_dict = {}

for target_SystemPath in target_SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, target_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, target_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()

### _k_<sub>_4_</sub>, _k_<sub>_6_</sub> を変更するための `EntityStub` を作成する

In [ ]:
R4 = createEntityStub( 'Process:/Cell:R4' )
R6 = createEntityStub( 'Process:/Cell:R6' )

### グラフにプロットするためのデータを格納する辞書 `Data_dict` を用意する

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData() )[ :, :2 ]

Variable_FullPN_list = [ FullPN for FullPN in Data_dict.keys() if FullPN.split(':')[ 0 ] == 'Variable' and FullPN.split(':')[ 3 ] == 'Value' ]
Process_FullPN_list = [ FullPN for FullPN in Data_dict.keys() if FullPN.split(':')[ 0 ] == 'Process' ]

### _k_<sub>_4_</sub>, _k_<sub>_6_</sub> を設定し、シミュレーションを実行、グラフを描画する関数 `test_k4_and_k6()` を定義する

In [ ]:
def test_k4_and_k6( k4, k6, step_width = 100 ):
    R4['k4'] = k4
    R6['k6'] = k6
    
    run( 10 * step_width )
    _t = getCurrentTime()

    for FullPN, Logger in Logger_dict.items():
        Data_dict[ FullPN ] = np.array( Logger.getData( _t - step_width, _t, 0.1 ) )[ :, :2 ]

    plt.figure()
    for FullPN, d in Data_dict.items():
        if FullPN in Variable_FullPN_list:
            plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

    plt.xlim( _t - step_width, _t + 40.0 )
    plt.legend( loc = 'upper right' )
    plt.title( 'k4 = {},  k6 = {}'.format( k4, k6 ) )

### 試行する _k_<sub>_4_</sub>, _k_<sub>_6_</sub> の値のセットを、リストとして用意する

In [ ]:
parameters_list = [
    [  30.0, 0.1  ],
    [  30.0, 0.25 ],
    [ 180.0, 0.25 ],
    [ 180.0, 1.0  ],
    [ 180.0, 2.0  ],
    [ 500.0, 2.0  ],
    [ 500.0, 1.3  ],
    [ 180.0, 1.3  ]
]

### for 文で parameters_list から _k_<sub>_4_</sub>, _k_<sub>_6_</sub> の値を取り出し、繰り返し処理でグラフを描き出す

In [ ]:
for p in parameters_list:
    test_k4_and_k6( p[0], p[1] )

### 全体を眺めてみる

In [ ]:
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), 0.1 ) )[ :, :2 ]

plt.figure( figsize = (15, 6) )
for FullPN, d in Data_dict.items():
    if FullPN in Variable_FullPN_list:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.xlim( 0, getCurrentTime() * 1.15 )
plt.legend( loc = 'upper right' )